## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Kavieng,PG,2021-12-06 03:26:21,-2.5744,150.7967,83.53,74,88,6.58
1,1,Bluff,NZ,2021-12-06 03:26:21,-46.6000,168.3333,57.79,56,8,21.39
2,2,Atuona,PF,2021-12-06 03:23:48,-9.8000,-139.0333,78.89,73,4,17.74
3,3,Castro,BR,2021-12-06 03:26:22,-24.7911,-50.0119,60.58,95,82,2.84
4,4,Hermanus,ZA,2021-12-06 03:26:22,-34.4187,19.2345,59.90,94,100,9.71


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Kavieng,PG,2021-12-06 03:26:21,-2.5744,150.7967,83.53,74,88,6.58
2,2,Atuona,PF,2021-12-06 03:23:48,-9.8000,-139.0333,78.89,73,4,17.74
14,14,San Patricio,US,2021-12-06 03:26:26,28.0170,-97.5169,75.88,85,100,14.32
15,15,Coahuayana,MX,2021-12-06 03:26:26,18.7333,-103.6833,80.08,93,100,1.95
17,17,Hithadhoo,MV,2021-12-06 03:26:27,-0.6000,73.0833,81.46,78,99,11.27
20,20,Pirapemas,BR,2021-12-06 03:26:28,-3.7267,-44.2233,75.06,92,80,2.21
22,22,Rikitea,PF,2021-12-06 03:26:29,-23.1203,-134.9692,76.89,73,96,7.02
24,24,Mumford,GH,2021-12-06 03:26:30,5.2618,-0.7590,79.47,86,26,7.76
26,26,Buala,SB,2021-12-06 03:26:31,-8.1450,159.5921,78.40,89,100,17.16
27,27,Ishigaki,JP,2021-12-06 03:26:31,24.3448,124.1572,75.33,57,40,16.11


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID       172
City          172
Country       172
Date          172
Lat           172
Lng           172
Max Temp      172
Humidity      172
Cloudiness    172
Wind Speed    172
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID       172
City          172
Country       172
Date          172
Lat           172
Lng           172
Max Temp      172
Humidity      172
Cloudiness    172
Wind Speed    172
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
# 5b. Create a new column "Hotel Name"
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Kavieng,PG,83.53,-2.5744,150.7967,
2,Atuona,PF,78.89,-9.8000,-139.0333,
14,San Patricio,US,75.88,28.0170,-97.5169,
15,Coahuayana,MX,80.08,18.7333,-103.6833,
17,Hithadhoo,MV,81.46,-0.6000,73.0833,
20,Pirapemas,BR,75.06,-3.7267,-44.2233,
22,Rikitea,PF,76.89,-23.1203,-134.9692,
24,Mumford,GH,79.47,5.2618,-0.7590,
26,Buala,SB,78.40,-8.1450,159.5921,
27,Ishigaki,JP,75.33,24.3448,124.1572,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City          172
Country       172
Max Temp      172
Lat           172
Lng           172
Hotel Name    172
dtype: int64

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))